In [42]:
import pandas as pd
import requests

# Task 1

## Question 1

In [98]:
# Replace 'path_to_your_excel_file' with the actual path to your Excel file
excel_file_path = 'Country-Code.xlsx'

# Read the Excel file into a DataFrame
country_df = pd.read_excel(excel_file_path)

# Display the DataFrame
# country_df

In [140]:
def fetch_json_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Error fetching data:", e)
        return None

url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
json_data = fetch_json_data(url)

if json_data:
    # Now you can work with the restaurant data dictionary
    print("Data successfully fetched!")
    # Example usage:
    print("Length of JSON:", len(json_data))
else:
    print("Failed to fetch data. Please check the URL or your internet connection.")


Data successfully fetched!
Length of JSON: 79


In [141]:
def process_one_page(restaurant_data):
    # Lists to store extracted data
    restaurant_ids = []
    restaurant_names = []
    countries = []
    cities = []
    user_rating_votes = []
    user_aggregate_ratings = []
    cuisines_list = []

    # Iterate over each restaurant data and extract required fields
    for restaurant_entry in restaurant_data:
        restaurant_info = restaurant_entry["restaurant"]
        restaurant_ids.append(restaurant_info['R']['res_id'])
        restaurant_names.append(restaurant_info['name'])
        countries.append(restaurant_info['location']['country_id'])
        cities.append(restaurant_info['location']['city'])
        user_rating_votes.append(int(restaurant_info['user_rating']['votes']))
        user_aggregate_ratings.append(float(restaurant_info['user_rating']['aggregate_rating']))
        cuisines_list.append(restaurant_info['cuisines'])

    # Create DataFrame
    restaurants_df = pd.DataFrame({
        'Restaurant Id': restaurant_ids,
        'Restaurant Name': restaurant_names,
        'Country': countries,
        'City': cities,
        'User Rating Votes': user_rating_votes,
        'User Aggregate Rating': user_aggregate_ratings,
        'Cuisines': cuisines_list
    })
    
    # Join both tables
    merged_df = pd.merge(restaurants_df, country_df, left_on='Country', right_on='Country Code', how='inner')
    # Pick columns
    df = merged_df[['Restaurant Id', 'Restaurant Name', 'Country_y', 'City', 'User Rating Votes', 'User Aggregate Rating', 'Cuisines']]
    final_df = df.rename(columns={'Country_y': 'Country'})
    
    return final_df
    

process_one_page(json_data[3]["restaurants"])

,Restaurant Id,Restaurant Name,Country,City,User Rating Votes,User Aggregate Rating,Cuisines
0,58882,Byg Brewski Brewing Company,India,Bangalore,17394,4.9,"Continental, North Indian, Italian, South Indi..."
1,18706428,Byg Brewski Brewing Company,India,Bangalore,5692,4.8,"Finger Food, Asian, Pizza, Italian, North Indi..."
2,18871246,ShakesBierre,India,Bangalore,572,4.5,"Asian, Continental, European, North Indian"
3,18772094,Bombay Adda,India,Bangalore,3023,4.3,Modern Indian
4,18578798,TBC Sky Lounge,India,Bangalore,7673,4.7,"Continental, Asian, Italian, North Indian"
5,18945224,Secret Of Bangalore,India,Bangalore,866,4.3,"North Indian, South Indian, Continental, Asian..."
6,18254597,Brew and Barbeque - A Microbrewery Pub,India,Bangalore,6330,4.6,"Continental, North Indian, BBQ, Steak"
7,18387426,The Black Pearl,India,Bangalore,7769,4.7,"North Indian, European, Mediterranean, BBQ"
8,57833,Big Pitcher,India,Bangalore,9664,4.6,"American, Continental, North Indian, Mediterra..."
9,54162,The Black Pearl,India,Bangalore,10892,4.8,"North Indian, European, Mediterranean"


In [151]:
def process_all_pages(json_data):
    # Initialize an empty list to store DataFrames
    dfs = []
    for item in json_data:
        df = process_one_page(item["restaurants"])
        dfs.append(df)
    concatenated_df = pd.concat(dfs)
    concatenated_df['Restaurant Id'] = concatenated_df['Restaurant Id'].astype(int)
    concatenated_df['User Rating Votes'] = concatenated_df['User Rating Votes'].astype(int)
    return concatenated_df

question_1_results = process_all_pages(json_data)

In [152]:
question_1_results.to_csv('restaurants.csv', index=False)

## Question 2

In [229]:
restaurants = json_data[0]["restaurants"]

In [230]:
len(restaurants)

20

In [238]:
def extract_events_for_one_page(restaurants):
    all_events = []
    for restaurant in restaurants:
        # Process one restaurant first!
        restaurant_data = restaurant["restaurant"]
        restaurant_id = restaurant_data["id"]
        restaurant_name = restaurant_data["name"]
        if "zomato_events" in restaurant_data.keys():
            zomato_events = restaurant_data["zomato_events"]
            restaurant_events = []
            for event in zomato_events:
                event_data = event["event"]
                event_id =  event_data["event_id"]
                photo_url = pd.NA
                if len(event_data["photos"]) != 0:
                    photo_url = event_data["photos"][0]["photo"]["url"]
                event_title = event_data["title"]
                event_start_date = event_data["start_date"]
                event_end_date = event_data["end_date"]
                restaurant_events.append({
                    "Event Id": event_id,
                    "Restaurant Id": restaurant_id,
                    "Restaurant Name": restaurant_name,
                    "Photo URL": photo_url,
                    "Event Title": event_title,
                    "Event Start Date": event_start_date,
                    "Event End Date": event_end_date
                })
            all_events.extend(restaurant_events)

    all_events_df = pd.DataFrame(all_events)
    return all_events_df
    
extract_events_for_one_page(json_data[0]["restaurants"])

,Event Id,Restaurant Id,Restaurant Name,Photo URL,Event Title,Event Start Date,Event End Date
0,322331,18649486,The Drunken Botanist,https://b.zmtcdn.com/data/zomato_events/photos...,BackToBasic Wednesdays !!\n\n\n,2019-03-06,2019-08-28
1,332812,308322,Hauz Khas Social,https://b.zmtcdn.com/data/zomato_events/photos...,Live 20/20 Match Screenings,2019-03-29,2019-05-23
2,336644,18856789,AIR- An Ivory Region,https://b.zmtcdn.com/data/zomato_events/photos...,Dhol Bhangra Night,2019-04-10,2019-04-11
3,336889,18382360,Local,<NA>,Cocktail Wednesday,2019-04-10,2019-04-10
4,336888,18382360,Local,<NA>,Cocktail Wednesday,2019-04-10,2019-04-10
5,336890,18382360,Local,https://b.zmtcdn.com/data/zomato_events/photos...,Cocktail Wednesday,2019-04-10,2019-04-10
6,336894,18382360,Local,https://b.zmtcdn.com/data/zomato_events/photos...,Hip Hop Friday,2019-04-12,2019-04-12
7,336903,18382360,Local,https://b.zmtcdn.com/data/zomato_events/photos...,Urban Saturday,2019-04-13,2019-04-13
8,116096,18466951,Jungle Jamboree,https://b.zmtcdn.com/data/zomato_events/photos...,Festive Food Bonanza || Special Buffet Prices ...,2019-01-21,2019-05-31
9,161417,18466951,Jungle Jamboree,https://b.zmtcdn.com/data/zomato_events/photos...,7 Course High Tea Birthday & Kitty Parties @ R...,2019-01-21,2019-05-31


In [259]:
def extract_all_events(json_data):
    # Initialize an empty list to store DataFrames
    dfs = []
    for item in json_data:
        df = extract_events_for_one_page(item["restaurants"])
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index = True)
    return concatenated_df

all_events = extract_all_events(json_data)

ValueError: No objects to concatenate

In [244]:
all_events

,Event Id,Restaurant Id,Restaurant Name,Photo URL,Event Title,Event Start Date,Event End Date
0,322331,18649486,The Drunken Botanist,https://b.zmtcdn.com/data/zomato_events/photos...,BackToBasic Wednesdays !!\n\n\n,2019-03-06,2019-08-28
1,332812,308322,Hauz Khas Social,https://b.zmtcdn.com/data/zomato_events/photos...,Live 20/20 Match Screenings,2019-03-29,2019-05-23
2,336644,18856789,AIR- An Ivory Region,https://b.zmtcdn.com/data/zomato_events/photos...,Dhol Bhangra Night,2019-04-10,2019-04-11
3,336889,18382360,Local,<NA>,Cocktail Wednesday,2019-04-10,2019-04-10
4,336888,18382360,Local,<NA>,Cocktail Wednesday,2019-04-10,2019-04-10
...,...,...,...,...,...,...,...
179,305850,7001670,"Eight - Cordis, Auckland Hotel",https://b.zmtcdn.com/data/zomato_events/photos...,Smoky Thursdays,2019-03-01,2019-04-30
180,298162,7001670,"Eight - Cordis, Auckland Hotel",https://b.zmtcdn.com/data/zomato_events/photos...,Chocolate Tuesdays,2019-04-05,2019-08-21
181,329300,18981544,Katsu YA,https://b.zmtcdn.com/data/zomato_events/photos...,Review us on Zomato,2019-03-22,2019-04-30
182,329301,18981544,Katsu YA,https://b.zmtcdn.com/data/zomato_events/photos...,Opening Promo,2019-03-22,2019-04-30


In [251]:
def filter_events(all_events):
    # Convert "Event Start Date" and "Event End Date" columns to datetime objects
    all_events["Event Start Date"] = pd.to_datetime(all_events["Event Start Date"])
    all_events["Event End Date"] = pd.to_datetime(all_events["Event End Date"])
    
    # Filter rows where event falls within April 2019
    april_2019_mask = (all_events["Event Start Date"].dt.month == 4) & (all_events["Event Start Date"].dt.year == 2019) | \
                      (all_events["Event End Date"].dt.month == 4) & (all_events["Event End Date"].dt.year == 2019)
    april_2019_events = all_events[april_2019_mask]
    
    return april_2019_events

filtered_events = filter_events(all_events)

In [252]:
filtered_events.to_csv('restaurant_events.csv', index=False)

## Question 3

In [279]:
def extract_ratings_for_one_page(restaurants):
    all_ratings = []  # Initialize an empty list to store DataFrames
    for restaurant in restaurants:
        ratings_data = restaurant["restaurant"]["user_rating"]
        aggregate_rating = ratings_data["aggregate_rating"]
        rating_text = ratings_data["rating_text"]

        # Create a DataFrame for each restaurant's ratings
        ratings_df = pd.DataFrame({
            "Aggregate Rating": [aggregate_rating],
            "Rating Text": [rating_text]
        })

        # Append the DataFrame to the list
        all_ratings.append(ratings_df)

    if len(all_ratings) == 0:
        return
        
    # Concatenate all DataFrames in the list into one DataFrame
    all_ratings_concatenated = pd.concat(all_ratings, ignore_index=True)

    return all_ratings_concatenated

extract_ratings_for_one_page(json_data[1]["restaurants"])

,Aggregate Rating,Rating Text
0,4.4,Very Good
1,4.5,Excellent
2,4.6,Excellent
3,4.7,Excellent
4,4.3,Very Good
5,4.2,Very Good
6,4.8,Excellent
7,4.4,Very Good
8,4.3,Very Good
9,4.8,Excellent


In [296]:
def extract_all_ratings(json_data):
    # Initialize an empty list to store DataFrames
    dfs = []
    for item in json_data:
        df = extract_ratings_for_one_page(item["restaurants"])
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index = True)
    return concatenated_df

all_ratings = extract_all_ratings(json_data)

In [298]:
def process_ratings(all_ratings):
    # Convert 'Aggregate Rating' column to float
    all_ratings['Aggregate Rating'] = all_ratings['Aggregate Rating'].astype(float)
    
    # Sort DataFrame by 'Aggregate Rating' in descending order
    all_ratings_sorted = all_ratings.sort_values(by='Aggregate Rating', ascending=False)
    
    # Remove duplicates
    all_ratings_sorted_no_duplicates = all_ratings_sorted.drop_duplicates()
    
    # Reset index
    all_ratings_sorted_no_duplicates.reset_index(drop=True, inplace=True)
    
    # Define the list of acceptable values for Rating Text
    acceptable_ratings = ['Excellent', 'Very Good', 'Good', 'Average', 'Poor']
    
    # Filter the DataFrame based on the acceptable values for Rating Text
    filtered_ratings = all_ratings_sorted_no_duplicates[all_ratings_sorted_no_duplicates['Rating Text'].isin(acceptable_ratings)]
    
    return filtered_ratings

# Example usage:
processed_ratings = process_ratings(all_ratings)
processed_ratings

,Aggregate Rating,Rating Text
0,4.9,Excellent
3,4.8,Excellent
5,4.7,Excellent
8,4.6,Excellent
9,4.5,Excellent
11,4.4,Very Good
13,4.3,Very Good
16,4.2,Very Good
17,4.1,Very Good
20,4.0,Very Good


| Rating Text | Aggregate Rating Range |
|-------------|------------------------|
| Excellent   | 4.5 - 5.0              |
| Very Good   | 4.0 - 4.4              |
| Good        | 3.5 - 3.9              |
| Average     | 3.0 - 3.4              |
| Poor        | Below 3.0              |


# Task 2